In [5]:
import numpy as np
import requests
import pandas as pd
import pandas as pd
import plotly.express as px


# Function to fetch rainfall data for given latitude and longitude
def fetch_rainfall_data(latitude, longitude):
    # Your API key from OpenWeatherMap
    api_key = '89c929e06a0dd8c891226c5934e3201e'

    # Construct the API URL
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={api_key}"

    # Make the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Initialize variable to store cumulative rainfall
        cumulative_rainfall = 0

        # Loop through the forecast data
        for forecast in data['list']:
            # Check if there's rainfall data available
            if 'rain' in forecast:
                cumulative_rainfall += forecast['rain'].get('3h', 0)  # Add rainfall if available

        # Divide the cumulative rainfall by 2 unless it exceeds 10, in which case set it to 5
        if cumulative_rainfall > 10:
            return 1
        else:
            return cumulative_rainfall / 10

    else:
        print("Error fetching data from the API.")
        return None

# Read CSV file with coordinates
csv_file = r"http://localhost:8081/fyp/2/backend/Backend_data/Testing_data.csv"
df = pd.read_csv(csv_file)

# Iterate over each row
for index, row in df.iterrows():
    latitude = row['Y']
    longitude = row['X']
    
    # Fetch rainfall data for the coordinates
    rainfall = fetch_rainfall_data(latitude, longitude)
    
    # Update the "rainfall_B" column with the fetched rainfall value
    df.at[index, 'Rainfall_B'] = rainfall

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_file, index=False)





LS_df = pd.read_csv("http://localhost:8081/fyp/2/backend/Backend_data/Training_data.csv")

numerical=LS_df.select_dtypes('number').columns
categorical = LS_df.select_dtypes('object').columns

# Count occurrences of -9999 in each column
missing_values_count = (LS_df == -9999).sum()
LS_df.replace(-9999, np.nan, inplace=True)

# Interpolate NaN values using spline interpolation
LS_df.interpolate(method='spline', order=5, inplace=True)

from sklearn.impute import SimpleImputer
# Replace -9999.0 with NaN
LS_df.replace(-9999.0, np.nan, inplace=True)

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # You can use other strategies like median or most_frequent
LS_df = pd.DataFrame(imputer.fit_transform(LS_df), columns=LS_df.columns)

X = LS_df.drop("Landslide_", axis=1)
y = LS_df.Landslide_
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

xtrain, xtest, ytrain, ytest =  train_test_split(X, y, test_size=0.25,random_state=42, shuffle=True)


from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=5, max_depth=3,random_state=350)
clf_rf.fit(xtrain, ytrain)
rf_pred = clf_rf.predict(xtest)

LS_df2 = pd.read_csv("http://localhost:8081/fyp/2/backend/Backend_data/Testing_data.csv")
LS_df2.replace(-9999, np.nan, inplace=True)
LS_df2.interpolate(method='spline', order=5, inplace=True)

LS_df2 = LS_df2.drop(columns=['X', 'Y', 'results'])


from sklearn.impute import SimpleImputer
# Replace -9999.0 with NaN
LS_df2.replace(-9999.0, np.nan, inplace=True)



# Handle missing values
imputer = SimpleImputer(strategy='mean')  # You can use other strategies like median or most_frequent
LS_df2 = pd.DataFrame(imputer.fit_transform(LS_df2), columns=LS_df2.columns)

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(LS_df2.select_dtypes(include=numerics).columns)
data = LS_df2[numerical_vars]

X = LS_df2


rf_pred = clf_rf.predict(X)



# Load your existing CSV file into a DataFrame
existing_df = pd.read_csv("http://localhost:8081/fyp/2/backend/Backend_data/Testing_data.csv")

# Assuming rf_pred is your DataFrame containing the results
# You may need to adjust the column names if they are different
rf_pred = pd.DataFrame(rf_pred, columns=['results'])

# Replace the existing 'results' column with the new values
existing_df['results'] = rf_pred['results']

# Save the modified DataFrame back to the CSV file
existing_df.to_csv("http://localhost:8081/fyp/2/backend/Backend_data/Testing_data.csv", index=False)


# Read CSV into pandas DataFrame
data = pd.read_csv("http://localhost:8081/fyp/2/backend/Backend_data/Testing_data.csv")

# Plot the coordinates using Plotly Express
fig = px.scatter_mapbox(data, lat="Y", lon="X", color="results",
                        color_discrete_sequence=["green", "red"], zoom=7)

# Customize map layout
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Show the plot
fig.show()